In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(
    urls=["https://criticalsoftware.com/en/legal"], mode="elements", strategy="fast",
)

docs = loader.load()

In [2]:
print(docs[10])

page_content='In most circumstances, your data will only be processed within the UK or EEA. If for any reason we need to have your data processed elsewhere we will inform you of that, any privacy risks involved and what we have done to minimise those risks. In some circumstances, we would also seek your consent for that.' metadata={'languages': ['eng'], 'filetype': 'text/html', 'parent_id': 'ccd6d67eab9f76cd1f66c082d39e1861', 'url': 'https://criticalsoftware.com/en/legal', 'category': 'NarrativeText'}


In [3]:
from langchain_experimental.text_splitter import SemanticChunker  
from langchain_community.embeddings import HuggingFaceEmbeddings 

embeddings = HuggingFaceEmbeddings() 

text_splitter = SemanticChunker(HuggingFaceEmbeddings())  
documents = text_splitter.split_documents(docs)

/tmp/ipykernel_328698/2043985166.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_328698/2043985166.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_328698/2043985166.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEm

In [4]:
for doc in documents:
    doc.metadata.pop("languages", None)
    doc.metadata.pop("category_depth", None)

In [5]:
documents[0]

Document(metadata={'filetype': 'text/html', 'url': 'https://criticalsoftware.com/en/legal', 'category': 'Title'}, page_content='Legal Information')

In [7]:
from langchain_milvus import Milvus
from pymilvus import connections

connections.connect("default", host="localhost", port="19530")

vector_store = Milvus.from_documents(documents,
                                     embeddings,
                                     collection_name="csw_legal",
                                     auto_id=True)